In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('./data/product_df.csv')

In [4]:
df= df[['Star Rating', 'Comment']]
df.shape

(1544, 2)

In [5]:
#Kiểm tra dữ liệu khuyết thiếu
print(df.isnull().sum())

Star Rating    0
Comment        0
dtype: int64


In [6]:
#Kiểm tra dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)

Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment
48              4                                   Rất tốt
76              5                                    Rất ok
107             2                             pin tụt nhanh
135             5                              sản phẩm tốt
173             2                             hao pin nhanh
177             5                              sản phẩm tốt
181             5                                    Rất ok
183             5                              sản phẩm tốt
192             5                               sản phẩm ok
205             5                               máy dùng ok
225             4                                       Tốt
227             5                               sản phẩm ok
237             5                               máy dùng ok
250             5                              sản phẩm tốt
426             4                              máy d

In [7]:
df = df.drop_duplicates(['Comment'])
df.shape

(1498, 2)

Làm sạch dữ liệu văn bản

In [8]:
import re

def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s.,!?]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def add_space_around_punctuation(text):
    # Tạo khoảng cách giữa từ và dấu câu
    text = re.sub(r'([.,!?])', r' \1 ', text)
    text = re.sub(r'\s{2,}', ' ', text)  # Loại bỏ khoảng trắng thừa
    return text

def clean_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    text = add_space_around_punctuation(text)
    return text

In [9]:
# Áp dụng hàm clean_text và thay thế cột 'comment'
df['Comment'] = df['Comment'].apply(clean_text)

# In DataFrame sau khi làm sạch
print(df)

      Star Rating                                            Comment
0               4                      điện thoại này dùng rất thích
1               4                               sử dụng thấy cũng ok
2               2                     bảo hành ít quá , chỉ 12 tháng
3               5                            sản phẩm mượt , chạy êm
4               3  cho mình hỏi muốn khởi động lại máy hay tắt ng...
...           ...                                                ...
1539            1  đem máy ra cho nhân viên xem mà về máy vẫn nón...
1540            3                       cũng được dù pin xuống nhanh
1541            1  hơi hối hận khi mua máy vội mà chưa tìm hiểu k...
1542            4  chụp ảnh ổn , điện tử ổn , màn hình đẹp , pin ...
1543            5  mua mới đã là ios 17 rồi nhé . nhân viên tư vấ...

[1498 rows x 2 columns]


In [10]:
#Loại bỏ stopwords
# Đọc dữ liệu từ file vietnamese-stopwords.txt
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [word.strip() for word in stopwords]

In [11]:
# Hàm để loại bỏ stopwords
def remove_stopwords(comment):
    words = comment.split()
    cleaned_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(cleaned_words)

In [12]:
# Áp dụng hàm và thay thế cột 'comment'
df['Comment'] = df['Comment'].apply(remove_stopwords)

# In DataFrame sau khi loại bỏ
print(df)

      Star Rating                                            Comment
0               4                               điện thoại rất thích
1               4                                         sử dụng ok
2               2                                      bảo hành , 12
3               5                            sản phẩm mượt , chạy êm
4               3                         khởi động máy tắt thao tác
...           ...                                                ...
1539            1                       đem máy nhân viên máy nóng ,
1540            3                                                pin
1541            1              hơi hối hận mua máy vội kĩ , máy chán
1542            4  chụp ảnh ổn , điện tử ổn , màn hình đẹp , pin ...
1543            5           mua ios 17 . nhân viên tư vấn nhiệt tình

[1498 rows x 2 columns]


Tách từ và mã hóa từ

In [13]:
from underthesea import word_tokenize
from nltk.probability import FreqDist
import pandas as pd

# Đảm bảo rằng bạn đã tải xuống tất cả các gói cần thiết từ nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vuhamy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
# Hàm tách từ và xây dựng từ vựng
def tokenize_and_build_vocab(comment):
    tokens = word_tokenize(comment)
    return tokens

# Áp dụng hàm vào cột 'Comment' của DataFrame
df['Tokenized_Comment'] = df['Comment'].apply(tokenize_and_build_vocab)

# Tính toán từ vựng từ các tokens
all_tokens = [token for sublist in df['Tokenized_Comment'] for token in sublist]
vocab = FreqDist(all_tokens)

# In kết quả
print("Tổng số từ duy nhất:", len(vocab))
print("Các từ và tần suất xuất hiện:")
for word, frequency in vocab.items():
    print(f"{word}: {frequency}")

Tổng số từ duy nhất: 2085
Các từ và tần suất xuất hiện:
điện thoại: 252
rất: 409
thích: 26
sử dụng: 127
ok: 62
bảo hành: 49
,: 1508
12: 16
sản phẩm: 125
mượt: 119
chạy: 48
êm: 12
khởi động: 29
máy: 678
tắt: 41
thao tác: 5
mua: 470
03: 1
tuần: 49
.: 1676
kết luận: 1
pin sạc: 2
thời pin: 2
tùy: 1
nhu cầu: 5
camera: 54
ổn: 165
phản hồi: 9
ứng dụng: 51
màu: 56
xanh: 12
đẹp: 161
nhẹ: 16
năng: 7
hỗ trợ: 37
5: 53
triệu: 36
2: 134
lỗi: 109
nóng: 96
tụt pin: 14
ko thể: 1
chấp máy: 1
chục: 2
củ: 9
mấy: 46
tiếng: 63
chán: 39
thất vọng: 42
1: 217
tr: 6
liên tục: 18
mặt: 21
lưng: 22
kính: 3
vs fe: 1
lắm: 83
nha: 14
tụt: 74
pin: 345
nóng máy: 23
cảm giác: 13
lo sợ: 1
cửa hàng không: 5
đổi: 81
tệ: 56
cũ: 25
ân hận: 1
không: 653
chụp: 97
ảnh: 42
hơi: 63
châm: 1
thực hối hận: 1
kinh khủng: 14
nóng tác: 1
vụ: 5
nhẹ nhàng: 2
chip: 3
exynos: 2
đợi: 9
update: 2
thiết kế: 14
hàng: 91
an tâm chất: 1
chất: 36
giá: 112
352024: 1
giới: 69
di động: 108
máy ok: 1
100: 18
nhân viên: 160
tư vấn: 70
hài sản phẩm: 11

In [15]:
from gensim.models import KeyedVectors

In [17]:
# Đường dẫn đến file mô hình Word2Vec của bạn
model_path = 'vi_word2vec.txt'
# Load mô hình từ file
w2v_model = KeyedVectors.load_word2vec_format(model_path)

In [18]:
# Hàm mã hóa từng từ trong câu thành vector từ mô hình Word2Vec
def encode_comment(comment):
    tokens = word_tokenize(comment)
    vectors = []
    for token in tokens:
        if token in w2v_model:
            vectors.append(w2v_model[token])
        else:
            # Nếu từ không có trong từ vựng của mô hình, xử lý tùy ý
            vectors.append([0] * w2v_model.vector_size)  # Chỉ định vector không
    return vectors

# Áp dụng hàm vào cột 'Comment' của DataFrame
df['Encoded_Comment'] = df['Comment'].apply(encode_comment)

# In kết quả
print(df[['Comment', 'Encoded_Comment']])

                                                Comment  \
0                                  điện thoại rất thích   
1                                            sử dụng ok   
2                                         bảo hành , 12   
3                               sản phẩm mượt , chạy êm   
4                            khởi động máy tắt thao tác   
...                                                 ...   
1539                       đem máy nhân viên máy nóng ,   
1540                                                pin   
1541              hơi hối hận mua máy vội kĩ , máy chán   
1542  chụp ảnh ổn , điện tử ổn , màn hình đẹp , pin ...   
1543           mua ios 17 . nhân viên tư vấn nhiệt tình   

                                        Encoded_Comment  
0     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
1     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
2     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
3     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
4